# Infosys Quarterly Report Analysis

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

In [1]:
!pip install getpass4 pypdf openai faiss-cpu llama-index llama-index-readers-file llama-index-vector-stores-faiss llama-index-embeddings-openai llama-index-llms-groq llama-index-embeddings-huggingface

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
import os
from getpass import getpass
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [4]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.node_parser import (SentenceWindowNodeParser,)
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Document
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [5]:
import faiss

## Configure LLM service

In [6]:
#llm = OpenAI(temperature=0,
#             model="chatgpt-4o-latest",
#             max_tokens=500)


llm = Groq(model="llama-3.3-70b-versatile")

In [44]:
embed_model = HuggingFaceEmbedding(model_name="intfloat/e5-large-v2")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [45]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

## Load data
Downloaded from

https://www.infosys.com/investors/reports-filings/quarterly-results.html

In [46]:
documents = [
    {
        "file_path": "/content/ifrs-inr-press-release_q1_2022.pdf",
        "metadata": {"year": "2022", "quarter": "Q1", "company": "Infosys"}
    },
    {
        "file_path": "/content/ifrs-inr-press-release_q1_2023.pdf",
        "metadata": {"year": "2023", "quarter": "Q1", "company": "Infosys"}
    },
    {
        "file_path": "/content/ifrs-inr-press-release_q1_2024.pdf",
        "metadata": {"year": "2024", "quarter": "Q1", "company": "Infosys"}
    }
]

In [47]:
nodes = []
parser = SimpleNodeParser()

for doc in documents:
    reader = SimpleDirectoryReader(input_files=[doc["file_path"]])
    data = reader.load_data()

    # Attach metadata
    for d in data:
        d.metadata = doc["metadata"]

    nodes.extend(parser.get_nodes_from_documents(data, show_progress=True))

print(f"Loaded {len(nodes)} nodes from documents")

Parsing nodes:   0%|          | 0/8 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/7 [00:00<?, ?it/s]

Loaded 21 nodes from documents


# Build indices

In [50]:
# dimensions of text-ada-embedding-002
d = 1024
faiss_index = faiss.IndexFlatL2(d)

In [51]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults()
all_index = VectorStoreIndex(nodes, storage_context=storage_context)

## Build query engines

In [55]:
q_engine = all_index.as_query_engine(similarity_top_k=5)

## Run queries

In [60]:
response = q_engine.query(
    "What is the QoQ revenue growth in Q1 FY24?"
)

In [61]:
print(response)

To calculate the QoQ revenue growth in Q1 FY24, we need the revenue data for Q1 FY24 and Q4 FY23. However, the provided data only includes the revenues for the 3 months ended June 30, 2021, and June 30, 2020. 

Since the new context does not provide the necessary data for Q1 FY24 and Q4 FY23, we cannot calculate the QoQ revenue growth. Therefore, we cannot provide a new answer based on the given information. 

However, given the instruction to repeat the answer when in doubt, the response would be: 

1.0%


In [66]:
response = q_engine.query("Can you compare the operating margins for first quaters in 3 years FY22, FY23 and FY24")

In [67]:
print(response)

To compare the operating margins for the first quarters in 3 years, we need to look at the data provided for each year.

For FY24, the operating margin for Q1 is 20.8%.
For FY23, the operating margin for Q1 is 20.1% (as per the press release dated July 24, 2022).
For FY22, we can find the operating margin for Q1 by looking at the statement of comprehensive income. The operating profit is `6,914 crore and the revenues are `34,470 crore. However, this data is for the quarter ended June 30, 2022, which is Q1 of FY23. To find the data for Q1 of FY22, we need to look at the statement of comprehensive income for the 3 months ended June 30, 2021, which is not provided. But we can see the operating profit and revenues for the 3 months ended June 30, 2021, and June 30, 2022. Using the data for June 30, 2021, and June 30, 2022, we can calculate the operating margin for Q1 of FY22 and FY23. The operating profit for Q1 of FY22 is `6,603 crore and the revenues are `27,896 crore. The operating margi